<a href="https://colab.research.google.com/github/TAKE-JP-17/Pytorch/blob/main/Instant_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Compute Trend for the whole time series
trend_mndwi = mndwi_collection.select(['system:time_start','mndwi_aoi']).reduce(ee.Reducer.linearFit())
trend_ndvi = ndvi_collection.select(['system:time_start','ndvi_aoi']).reduce(ee.Reducer.linearFit())
trend_ndbi = ndbi_collection.select(['system:time_start','ndbi_aoi']).reduce(ee.Reducer.linearFit())

#MNDWI
# 提取斜率
slope_mndwi = trend_mndwi.select('scale')

# 計算斜率的最小值和最大值
min_slope_mndwi = ee.Number(slope_mndwi.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=shp,
    scale=30,
    maxPixels=1e13
).get('scale'))

max_slope_mndwi = ee.Number(slope_mndwi.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=shp,
    scale=30,
    maxPixels=1e13
).get('scale'))

# 正規化斜率到-1到1之間
normalized_slope_mndwi = slope_mndwi.subtract(min_slope_mndwi).divide(max_slope_mndwi.subtract(min_slope_mndwi)).multiply(2).subtract(1).multiply(900)


# 合成正規化後的趨勢影像
normalized_trend_mndwi = trend_mndwi.addBands(normalized_slope_mndwi.rename('normalized_scale'), overwrite=True)

# 可視化正規化後的趨勢圖
map.addLayer(normalized_trend_mndwi, {'min': -900, 'max': 900, 'bands': ['normalized_scale'], 'palette': ['#d7191c','#fdae61','#ffffbf','#abd9e9','#2c7bb6']}, 'Normalized Trend_MNDWI')